In [1]:
import random

def selectref(dicref,el):
    return random.choice(dicref[el])

dicref={'U':['T'],
'R':['A','G'],
'Y':['C','T'],
'S':['G','C'],
'W':['A','T'],
'K':['G','T'],
'M':['A','C'],
'B':['C','G','T'],
'D':['A','G','T'],
'H':['A','C','T'],
'V':['A','C','G'],
'N':['A','T','G','C'],
'A':['A'],
'C':['C'],
'G':['G'],
'T':['T']}

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D,Reshape, LSTM, Dropout, TimeDistributed
from tensorflow.keras.callbacks import EarlyStopping
import pickle
from sklearn.utils import shuffle

bases={'A':np.array([0,0,0,1]), 'C':np.array([0,0,1,0]), 'G':np.array([0,1,0,0]), 'T':np.array([1,0,0,0]),'NA':['0','0','0','0']};

#helper function to encode input 
def one_hot_encode(seq, unique_elems,minlen):
    #slice first few bases in the sequence
    seq=seq[:minlen]
    seq_encoded=[]
    for el in seq:
        #select the appropriate 
        seq_encoded.append(bases[selectref(dicref,el)])
        
    seq=np.array(seq_encoded)

    return np.reshape(seq,(seq.shape[0],seq.shape[1],1))

def one_hot_encode_2(y,num_classes):
    y_encoded=[]
    for value in y:
	    letter = [0 for _ in range(num_classes)]
	    letter[value] = 1
	    y_encoded.append(letter)
    return np.array(y_encoded,dtype=np.float16)


!pip install BioPython

from Bio import SeqIO

f=open("unique_elems.pkl","rb")
unique_elems=pickle.load(f)
unique_elems=dict()
x_data=[]
y_data=[]
minlen=302


#Enter path to file 
for sequence in SeqIO.parse('path to file', "fasta"):
    x_data.append(one_hot_encode(sequence.seq,bases,minlen))
    y_data.append(unique_elems[sequence.description.split()[1]])

y_data=one_hot_encode_2(y_data,len(unique_elems))



In [ ]:
import os
checkpoint_path = "/content/drive/My Drive/training_genomics_lstm/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [ ]:
from tensorflow.keras.layers import Input
model=Sequential()
model.add(Input(shape=(minlen,4,1)))
model.add(TimeDistributed(Conv1D(filters=16, kernel_size=3, activation='relu')))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(units=256))
model.add(Dense(units=lm, activation='softmax'))
# model.summary()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.load_weights("/content/drive/My Drive/training_genomics_lstm/cp.ckpt")


In [ ]:

for i in range(len(x_data)):
  pred=model.predict(x_data[i])